In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! pip install sacrebleu
! pip install googletrans
! pip install goslate

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 17.1MB 201kB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 921kB 30.7MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 102kB 14.9MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=612fa717c0a8331b1814c63de3e406dccbead045bce15da6fd18b77a02155109
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=ef8fa047adb92f6b26573af76bae9f1998a05640cb5a223226bbedcb0666d414
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletra

In [ ]:
%cd  '/content/drive/My Drive/Shopee Code League/Product Translation'
!ls

/content/drive/.shortcut-targets-by-id/1v9o97UbhvDNUvhaLciCCxeaD-4Zep2RJ/Shopee Code League/Product Translation
data  dev.csv  EDA.ipynb  google_trans.ipynb  old_data	submiss  submission.csv


In [ ]:
import os
import pandas as pd
import numpy as np
import random

import re
import urllib
from urllib.error import HTTPError
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from typing import List

import regex
from sacrebleu import corpus_bleu
from googletrans import Translator
from goslate import Goslate

tqdm.pandas()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# product_title_translation_eval_script.py
"""Sample evaluation script for product title translation."""

OTHERS_PATTERN = regex.compile(r'\p{So}')


def eval(preds: List[str], refs: List[str]) -> float:
    """BLEU score computation.

    Strips all characters belonging to the unicode category "So".
    Tokenize with standard WMT "13a" tokenizer.
    Compute 4-BLEU.

    Args:
        preds (List[str]): List of translated texts.
        refs (List[str]): List of target reference texts.
    """
    preds = [OTHERS_PATTERN.sub(' ', text) for text in preds]
    refs = [OTHERS_PATTERN.sub(' ', text) for text in refs]
    return corpus_bleu(
        preds, [refs],
        lowercase=True,
        tokenize='13a',
        use_effective_order= False
    ).score, preds, refs

In [ ]:
data_dir = './old_data/'
for fn in os.listdir(data_dir):
    print(os.path.join(data_dir,fn))

./old_data/train_tcn.csv
./old_data/train_en.csv
./old_data/test.csv
./old_data/train_tcn.csv.zip
./old_data/train_en.csv.zip


In [ ]:
data_dir = './data/'
for fn in os.listdir(data_dir):
    print(os.path.join(data_dir,fn))

./data/train_tcn.csv
./data/train_en.csv
./data/train_tcn.csv.zip
./data/dev_en.csv
./data/train_en.csv.zip
./data/test_tcn.csv
./data/dev_tcn.csv
./data/translated_test.csv
./data/translated_test.gsheet
./data/translated_dev.csv
./data/translated_dev.gsheet
./data/dev.csv


In [ ]:
train_en = pd.read_csv('./data/train_en.csv')
train_tcn = pd.read_csv('./data/train_tcn.csv')

dev_en = pd.read_csv('./data/dev_en.csv')
dev_tcn = pd.read_csv('./data/dev_tcn.csv')
dev_data = pd.read_csv('./data/dev.csv')

translated_test = pd.read_csv('./data/translated_test.csv')
test = pd.read_csv('./data/test_tcn.csv')
print(len(train_tcn), len(dev_data), len(test))

500000 1000 10000


In [ ]:
train_en.head()

,product_title,category
0,Recollections Color Splash Clear Stamps & Stencil,Hobbies & Stationery
1,"soap,lotion scrub set 400",Health & Personal Care
2,Spigen Galaxy S10e Case Tough Armor Gunmetal,Mobile Accessories
3,Acrylic Lanalon Bright Red,Hobbies & Stationery
4,303 FLAT SHEET/Blanket 100% cotton,Home & Living


In [ ]:
train_tcn.head()

,product_title,category
0,Gucci Gucci Guilty Pour Femme Stud Edition 罪愛女...,Health & Beauty
1,（二手）PS4 GTA 5 俠盜獵車手5 Grand Theif Auto V繁體 中文版,Game Kingdom
2,百獸卡,Life & Entertainment
3,nac nac活氧全效柔衣素,Mother & Baby
4,#Nike耐吉官方F.C. 男子足球長褲新款標準型 拒水 拉鏈褲腳\nCD0557,Men's Apparel


In [ ]:
test.head()

,text,split
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,public
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public
2,粉晶六角柱純銀項鍊,public
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,public
4,燈專屬優惠 *4盒,public


In [ ]:
translated_test.head()

,text,split,translated
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,public,"[PolarStar] Merino Wool Thermal Socks ""Light G..."
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public,Sweet Crystal ~ Natural Crystal Five Elements ...
2,粉晶六角柱純銀項鍊,public,Pink crystal hexagonal pillar sterling silver ...
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,public,3M SCOTCH VHB super strong double-sided adhesi...
4,燈專屬優惠 *4盒,public,Lamp exclusive discount *4 boxes


In [ ]:
SITE = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"
HEADER = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

{"English":"en",
 "Vietnamese":"vi",
 "Chinese":"zh"
 }

class TinyTranslator:
    def __init__(self, src='en', des='zh'):
        self.src = src
        self.des = des 
        self.ggtrans = Translator()
        self.goslate = Goslate()
        self.http_error = False

    def translate(self, query):

        if not self.http_error:
            try:
                output = self.crawl_trans(query)
            except urllib.error.HTTPError as e:
                print(e)
                self.http_error = True
                self.src = 'zh-cn' if self.src=='zh' else self.src 
                output = self.google_trans(query)
        else:
            try:
                output = self.google_trans(query)
            except Exception as e:
                print(e)
        
        return output
    
    def crawl_trans(self, query):
            link = SITE % (self.des, self.src, urllib.request.pathname2url(query))
            # print(link)

            req = urllib.request.Request(link, headers=HEADER)
            page = urllib.request.urlopen(req) 
            soup = BeautifulSoup( page, 'lxml')    
            output = soup.find('div', {"class":"t0"}).get_text()   

            return output 
    
    def google_trans(self, query):
        
        output = self.ggtrans.translate(query, dest=self.des, src=self.src).text
        return output
        
    def go_trans(self,query):
        
        output = self.goslate.translate(query, target_language=self.des, source_language=self.src)
        
        return output

In [ ]:
# en2ch = TinyTranslator('en', 'zh')
ch2en = TinyTranslator('zh', 'en')
# en2vi = TinyTranslator('en', 'vi')
# ch2vi = TinyTranslator('zh', 'vi')

# test
ch2en.translate("甜蜜水晶~天然水晶五行珠手鍊10mm手鍊")

In [ ]:
# fig, _ = plt.subplots(2,1, figsize=(20,15))
# plt.subplot(2,1,1)
# plt.title('English')
# ax = sns.countplot(train_en.category, order=train_en.category.value_counts().index)
# ax.set_xticklabels(labels=train_en.category.value_counts().index, rotation=50)
# plt.subplot(2,1,2)
# plt.title('Chinese')
# ax = sns.countplot(train_tcn.category, order=train_tcn.category.value_counts().index)
# ax.set_xticklabels(labels=train_tcn.category.value_counts().index, rotation=50)
# plt.show()

In [ ]:
dev_data.head()

,text,translated,ground_truth,split
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,OPPO A75 A75s A73 mobile phone shell soft shel...,Oppo A75 A75S A73 Phone Case Soft Rabbit Silic...,private
1,SOFT 99 鍍膜車蠟(強力撥水型),SOFT 99 Coated Car Wax (Strong Water Repellent...,SOFT 99 Coating Car Wax Strong Water Watt,private
2,低糖芒果乾 250g 臻御行,Low-sugar dried mango 250g Zhenyuxing,Low Sugar Mango Dry 250g Be The Royal,private
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,＊Trail culture ＊ROUND TOP space craft-diamond ...,* the culture Japan Imported Round Top Space C...,private
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,Hello Kitty Hello Kitty Kitty Sandals Kids Sho...,Hello Kitty Sandals Shoes White/Red Children n...,private


In [ ]:
def post_process(text):
    text = text.lower()    
    
    text = re.sub('\[', '【 ', text)
    text = re.sub('\]', ' 】', text)
    # text = re.sub('\[', '【', text)
    # text = re.sub('【', '【 ', text)
    # text = re.sub('\]', '】', text)
    # text = re.sub('】', ' 】', text)
    pattern = re.compile("【(.+\-)")
    matches = re.findall("【.+\-.+】",text)
    # for pattern in matches:
    #     new_pattern = old.replace("【", "【 ").replace("】", " 】")
    #     text = text.replace(pattern, new_pattern)
    #     print(text)
    text = re.sub('＊', '*', text)
    text = re.sub('\*', ' * ', text)
    text = text.replace('-', ' - ')
    text = text.replace('~', ' ~ ')
    text = text.replace('/', ' / ')
    text = text.replace('’', ' ’ ')
    text = text.replace('&', ' & ')
    text = text.replace('#', ' # ')
    text = text.replace('_', ' _ ')
    text = text.replace("men's", "men")
    text = text.replace(" pc", "pcs")
    # text = text.replace(" ml", "ml")    # lower
    
    # text = text.replace("women", "female")    # lower
    # text = re.sub(r'\d+ (g)', 'g', text)
    text = re.sub('\s+', ' ', text)
    # text = re.sub('\(', '( ', text)    # lower
    # text = re.sub('\)', ' )', text)
    return text
post_process('* 1.5 g iFairies Shoulder Messenger Bag Side Backpack Shoulder Bag [49146]  Lt - 234')    

' * 1.5 g ifairies shoulder messenger bag side backpack shoulder bag 【 49146 】 lt - 234'

In [ ]:
texts = dev_data.text.tolist()
score, preds, refs = eval(preds=dev_data.translated.apply(post_process), refs=dev_data.ground_truth)
score

41.92138399000379

In [ ]:
# for i in range(400, 500):
#     print('_'*80)
#     text = texts[i]
#     pred = preds[i]
#     ref = refs[i]
#     print(i)
#     print(f"Orginal text: {text}")
#     print(f"Prediction: {dev_data.iloc[i].translated}")
#     print(f"Processed prediction: {pred}")
#     print(f"Ground truth: {ref}")
#     print(f"Score: {eval([pred], [ref])[0]}")

In [ ]:
submiss = pd.DataFrame({"translation_output":translated_test["translated"].progress_apply(post_process)})
submiss.to_csv('submiss.csv', index=False)
submiss.head()

,translation_output
0,"【 polarstar 】 merino wool thermal socks ""light..."
1,sweet crystal ~ natural crystal five elements ...
2,pink crystal hexagonal pillar sterling silver ...
3,3m scotch vhb super strong double - sided adhe...
4,lamp exclusive discount * 4 boxes
